## Лабораторная работа №1

Замечание: при решении данных задач не подразумевается использования циклов или генераторов Python, если в задании не сказано обратного. Решение должно опираться на использования функционала библиотеки `numpy`.

1. Файл `minutes_n_ingredients.csv` содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива `numpy` типа `int32`, используя `np.loadtxt`. Выведите на экран первые 5 строк массива.

In [234]:
import numpy as np

my_data = np.loadtxt('data\minutes_n_ingredients.csv', dtype='int32', delimiter=",", skiprows=1)
print(my_data[0:5])

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 [ 67660      5      6]
 [157911     60     14]]


2. Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [213]:
print(np.mean(my_data, axis=0)[1])
print(np.mean(my_data, axis=0)[2])
print(np.min(my_data, axis=0)[1])
print(np.min(my_data, axis=0)[2])
print(np.max(my_data, axis=0)[1])
print(np.max(my_data, axis=0)[2])
print(np.median(my_data, axis=0)[1])
print(np.median(my_data, axis=0)[2]) 


21601.00169
9.05528
0
1
2147483647
39
40.0
9.0


3. Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля $q_{0.75}$. 

In [214]:
q = np.quantile(my_data[:,1], q=0.75)
my_data[:, 1] = my_data[:, 1].clip(max=q)
print(my_data)
len(my_data)

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 ...
 [498432     65     15]
 [370915      5      4]
 [ 81993     65     14]]


100000

4. Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [215]:
print(np.sum(my_data[:,1]==0))
print(my_data[411, :])
# my_data[my_data[:,1]==0] = 1
my_data[:,1][my_data[:,1]==0 ] = 1
print(my_data[411, :])

479
[9325    0   10]
[9325    1   10]


5. Посчитайте, сколько уникальных рецептов находится в датасете.

In [232]:
len(np.unique(my_data, axis=1))

100000

6. Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [217]:
print(len(np.unique(my_data[:,2])))
print(np.unique(my_data[:,2]))

37
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 39]


7. Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [218]:
data_ingredients_5 = my_data[my_data[:,2] <= 6]
print(data_ingredients_5)

[[446597     15      5]
 [204134      5      3]
 [ 25623      6      4]
 ...
 [ 52088     60      5]
 [128811     15      4]
 [370915      5      4]]


8. Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [223]:
print(max(my_data[:,2]/my_data[:,1]))

24.0


9. Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [225]:
np.mean(my_data[my_data[:, 1].argsort()][::-1][0:100][:,2])

9.96

10. Выберите случайным образом и выведите информацию о 10 различных рецептах

In [226]:
for _ in range(10):
    print(my_data[np.random.choice(my_data[:,1])])

[192462     65     11]
[192462     65     11]
[69317    10     8]
[69317    10     8]
[192462     65     11]
[25623     6     4]
[498382     65     18]
[504666     65     20]
[378114     20     11]
[498382     65     18]


11. Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [227]:
print(len(my_data[my_data[:,2] < np.mean(my_data[:,2])]) / len(my_data[:,2]) * 100, '%') 

58.802 %


12. Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.

In [229]:
result = (my_data[:,1]<=20) & (my_data[:,2]<=5).astype('int32')
arr = np.insert(my_data, 3, result, axis=1)
print(arr)

[[127244     60     16      0]
 [ 23891     25      7      0]
 [ 94746     10      6      0]
 ...
 [498432     65     15      0]
 [370915      5      4      1]
 [ 81993     65     14      0]]


13. Выведите процент "простых" рецептов в датасете

In [230]:
print(len(arr[arr[:,3] == 1])/ len(arr) * 100, '%')

9.552 %


14. Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.

In [231]:
short = my_data[my_data[:,1]<10] 
medium = my_data[(my_data[:,1]>=10) & (my_data[:,1]<20)]
long = my_data[my_data[:,1]>20]
max_size = min(map(len, [short,medium, long]))
a = np.vstack(([short[:max_size,:]], [medium[:max_size,:]], [long[:max_size,:]]))
print(a.shape)

(3, 7588, 3)
